# Install and load libraries

In [ ]:
#!pip install mlend
#!pip install -U scikit-image

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spkit as sp
sp.__version__

In [ ]:
import mlend
from mlend import download_yummy_small, yummy_small_load
mlend.__version__

In [ ]:
# from skimage import exposure
# from skimage.color import rgb2hsv, rgb2gray
# import skimage as ski
# ski.__version__

# Download data

In [ ]:
baseDir = download_yummy_small(save_to = '../MLEnd')
baseDir

# Create Train-Test Set

In [ ]:
TrainSet, TestSet, Map = yummy_small_load(datadir_main=baseDir,train_test_split='Benchmark_A')

In [ ]:
TrainSet.keys()

In [ ]:
TestSet.keys()

In [ ]:
Map

In [ ]:
TrainSet['Y'], TrainSet['Y_encoded']

In [ ]:
TrainSet['Y_encoded']

In [ ]:
X_train_paths = TrainSet['X_paths']
X_test_paths  = TestSet['X_paths']

Y_train = TrainSet['Y_encoded']
Y_test  = TestSet['Y_encoded']

In [ ]:
Y_train

# Visualize a few images

In [ ]:
Chips_Img = np.array(X_train_paths)[Y_train==0]
Rice_Img = np.array(X_train_paths)[Y_train==1]

In [ ]:
print('Rice')
plt.figure(figsize=(15,5))
for k,file in enumerate(Rice_Img[:5]):
    I = plt.imread(file)
    plt.subplot(1,5,k+1)
    plt.imshow(I)
    plt.axis('off')

plt.tight_layout()
plt.show()

print('Chips')
plt.figure(figsize=(15,5))
for k,file in enumerate(Chips_Img[:5]):
    I = plt.imread(file)
    plt.subplot(1,5,k+1)
    plt.imshow(I)
    plt.axis('off')

plt.tight_layout()
plt.show()

# Resizing Images

As it can be seen that images are not of same size, so first step is to resize all the images to one-fixed dimention.

To keep the aspect ratio of image as it is, we will append black color to make is image square and then resize to 200x200.

You could choose any other size or approach to resize images

In [ ]:
def equalise_img(I):
    N,M,C = I.shape
    Is = [exposure.equalize_hist(I[:,:,i]) for i in range(C)]
    #img_adapteq = exposure.equalize_adapthist(img, clip_limit=0.03)
    return np.array(Is).transpose([1,2,0])

def make_it_square(I, pad=0):
    N,M,C = I.shape
    if N>M:
        Is = [np.pad(I[:,:,i], [(0,0),(0, N-M)], 'constant', constant_values=pad) for i in range(C)]
    else:
        Is = [np.pad(I[:,:,i], [(0, M-N),(0,0)], 'constant', constant_values=pad) for i in range(C)]

    return np.array(Is).transpose([1,2,0])

def resize_img(I,size=[100,100]):
    N,M,C = I.shape
    Ir = [sp.core.processing.resize(I[:,:,i],size) for i in range(C)]
    return np.array(Ir).transpose([1,2,0])

In [ ]:
X_train = []
for k,file in enumerate(X_train_paths):
    sp.utils.ProgBar_JL(k,len(X_train_paths),L=50,color='blue')
    I = plt.imread(file)
    #I = equalise_img(I.copy())
    I = make_it_square(I, pad=0)
    I = resize_img(I,size=[200,200])
    X_train.append(I)


X_test = []
for k,file in enumerate(X_test_paths):
    sp.utils.ProgBar_JL(k,len(X_test_paths),L=50,color='blue')
    I = plt.imread(file)
    #I = equalise_img(I.copy())
    I = make_it_square(I, pad=0)
    I = resize_img(I,size=[200,200])
    X_test.append(I)

X_train = np.array(X_train)
X_test = np.array(X_test)
X_train.shape, X_test.shape

In [ ]:
plt.figure(figsize=(10,6))
for k,I in enumerate(X_train):
    plt.subplot(3,5,k+1)
    plt.imshow(I)
    plt.axis('off')
    k+=1
    if k>=15:break
plt.tight_layout()
plt.show()

# Feature Extraction

In [ ]:
# from skimage.transform import integral_image
# from skimage.feature import haar_like_feature
# from skimage.feature import haar_like_feature_coord
# from skimage.feature import draw_haar_like_feature
from skimage.feature import fisher_vector, ORB, learn_gmm
from skimage.feature import graycomatrix, graycoprops

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [ ]:
def get_yellow_component(I,t1=27, t2=33):
    Ihsv = (rgb2hsv(I)*255).astype('uint8')
    mask = (Ihsv[:,:,0]<t2)*(Ihsv[:,:,0]>t1)
    Ypx = mask.sum()
    return Ypx

def ORBFeatures(I):
    Ig = rgb2gray(I)
    detector_extractor = ORB(n_keypoints=5, harris_k=0.01)
    detector_extractor.detect_and_extract(Ig)
    fx = detector_extractor.descriptors.astype('float32')
    return fx


def GMLC_features(I):
    Ig = (rgb2gray(I)*255).astype('uint8')
    glcm = graycomatrix(Ig, distances=[5], angles=[0], levels=256,
                        symmetric=True, normed=True)
    f1 = graycoprops(glcm, 'dissimilarity')[0, 0]
    f2 = graycoprops(glcm, 'correlation')[0, 0]
    return f1,f2



def showConfMat(CM, labels = ['Chips','Rice']):
    plt.matshow(CM,cmap='Blues')
    for i in range(CM.shape[0]):
        for j in range(CM.shape[1]):
            plt.text(i,j,CM[i,j].round(2),ha='center',)
    plt.xticks([0,1],labels)
    plt.yticks([0,1],labels)
    plt.show()

In [ ]:
X_train_f = []
for k, I in enumerate(X_train):
    f1 = get_yellow_component(I)
    f2,f3 = GMLC_features(I)
    X_train_f.append([f1,f2,f3])

X_test_f = []
for k, I in enumerate(X_test):
    f1 = get_yellow_component(I)
    f2,f3 = GMLC_features(I)
    X_test_f.append([f1,f2,f3])

X_train_f = np.array(X_train_f)
X_test_f = np.array(X_test_f)
X_train_f.shape, X_test_f.shape

# Normalization

In [ ]:
MEAN = X_train_f.mean(0)
SD = X_train_f.std(0)

X_train_fn = (X_train_f - MEAN)/SD
X_test_fn = (X_test_f - MEAN)/SD

# Linear Model

In [ ]:
model = LinearSVC(C=1)
#model = RandomForestClassifier(n_estimators=5,max_depth=3)
model.fit(X_train_fn, Y_train)

In [ ]:
ytp = model.predict(X_train_fn)
ysp = model.predict(X_test_fn)

In [ ]:
train_accuracy = np.mean(ytp==Y_train)
test_accuracy  = np.mean(ysp==Y_test)

print('Training Accuracy\t',train_accuracy)
print('Testing  Accuracy\t',test_accuracy)

In [ ]:
Ac = np.mean(ysp[Y_test.astype(int)==0]==0)
Ar = np.mean(ysp[Y_test.astype(int)==1]==1)

Mc = np.mean(ysp[Y_test.astype(int)==0]==1)
Mr = np.mean(ysp[Y_test.astype(int)==1]==0)

In [ ]:
CM = np.array([[Ac, Mc],[Mr, Ar]])

In [ ]:
showConfMat(CM)